In [ ]:
# 7/22/21: https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython

%load_ext autoreload
%autoreload 2

# end cite


In [ ]:
import json
import os
from os.path import join, exists

import pandas as pd
import matplotlib.pyplot as plt

from utils import load_models, transformers_bert_completions, split_gen
from utils_child import child_models
import config

In [ ]:

paths = [
    #split_gen.get_split_folder('child', 'Alex', config.model_dir),
    './experiments/no_versioning/models/all/all/',
]

has_tags_list = [
    True,
]

for path, tags in zip(paths, has_tags_list):

    tag_str =  "with_tags" if tags else "no_tags"

    with open(join(path, join(tag_str, 'trainer_state.json')), 'r') as f:

        gen_dict = json.load(f)
        this_history = gen_dict['log_history']

        epochs = []; losses = []

        for step_dict in this_history[:-1]: # Omit summary stats
            if 'eval_loss' not in step_dict: continue
            epochs.append(step_dict['epoch'])
            losses.append(step_dict['eval_loss'])

        train_epochs = []; train_losses = []
        for step_dict in this_history[:-1]: # Omit summary stats
            if 'loss' not in step_dict: continue
            train_epochs.append(step_dict['epoch'])
            train_losses.append(step_dict['loss'])

        print(gen_dict) # Because it's still on the debugging mode.

        this_step = gen_dict['best_model_checkpoint'].split('/')[-1]
        dataset_name = '/'.join(path.split('/')[-2:]) + f'/{tag_str} | step: {this_step}' + f"/{gen_dict['global_step']}"

    plt.title(f'{dataset_name}')
    plt.xlabel('epoch'); plt.ylabel('loss')
    plt.plot(epochs, losses, label = 'val')
    plt.plot(train_epochs, train_losses, label = 'train')

    plt.legend(); plt.show()
